<a href="https://colab.research.google.com/github/Tclack88/misc-analysis/blob/main/mandarin_frequency_analysis/pinyin_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Using this to provide example word for the heisig deck

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [10]:
url_start = "https://learn-chinese-words.com/dictionary.php?input="
source = requests.get(url_start+"弟").text
soup = BeautifulSoup(source, 'html.parser')
results = soup.find('table', {'class':'results'})
results

<table class="results">
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/伟大">伟大</a></td><td>wei3 da4 </td><td>great</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/兄弟">兄弟</a></td><td>xiong1 di4 </td><td>brotherly</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/弟">弟</a></td><td>di4 </td><td>variant of 悌[ti4]</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/弟兄">弟兄</a></td><td>di4 xiong1 </td><td>comrade</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/弟弟">弟弟</a></td><td>di4 di5 </td><td>younger brother</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/内弟">内弟</a></td><td>nei4 di4 </td><td>wife's younger brother</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/子弟">子弟</a></td><td>zi3 di4 </td><td>people</td></tr>
<tr><td><a href="https://learn-chinese-words.com/detail/chinese/伟">伟</a></td><td>wei3 </td><td>big</td></tr>
<tr><td

In [38]:
df = pd.read_html(str(results))[0]
df

,0,1,2
0,伟大,wei3 da4,great
1,兄弟,xiong1 di4,brotherly
2,弟,di4,variant of 悌[ti4]
3,弟兄,di4 xiong1,comrade
4,弟弟,di4 di5,younger brother
5,内弟,nei4 di4,wife's younger brother
6,子弟,zi3 di4,people
7,伟,wei3,big
8,弟子,di4 zi3,disciple
9,宏伟,hong2 wei3,grand


In [29]:
# lifted from here https://stackoverflow.com/questions/8200349/convert-numbered-pinyin-to-pinyin-with-tone-marks
import re

pinyinToneMarks = {
    'a': 'āáǎà', 'e': 'ēéěè', 'i': 'īíǐì',
    'o': 'ōóǒò', 'u': 'ūúǔù', 'ü': 'ǖǘǚǜ',
    'A': 'ĀÁǍÀ', 'E': 'ĒÉĚÈ', 'I': 'ĪÍǏÌ',
    'O': 'ŌÓǑÒ', 'U': 'ŪÚǓÙ', 'Ü': 'ǕǗǙǛ'
}

def convertPinyinCallback(m):
    tone=int(m.group(3))%5
    r=m.group(1).replace('v', 'ü').replace('V', 'Ü')
    # for multple vowels, use first one if it is a/e/o, otherwise use second one
    pos=0
    if len(r)>1 and not r[0] in 'aeoAEO':
        pos=1
    if tone != 0:
        r=r[0:pos]+pinyinToneMarks[r[pos]][tone-1]+r[pos+1:]
    return r+m.group(2)

def convertPinyin(s):
    return re.sub(r'([aeiouüvÜ]{1,3})(n?g?r?)([012345])', convertPinyinCallback, s, flags=re.IGNORECASE)

print(convertPinyin('Ni3 hao3 ma0?'))

Nǐ hǎo ma?


In [45]:
df[1] = df[1].apply(convertPinyin)
df[2] = df[2].apply(convertPinyin)
df

,0,1,2
0,伟大,wěi dà,great
1,兄弟,xiōng dì,brotherly
2,弟,dì,variant of 悌[tì]
3,弟兄,dì xiōng,comrade
4,弟弟,dì di,younger brother
5,内弟,nèi dì,wife's younger brother
6,子弟,zǐ dì,people
7,伟,wěi,big
8,弟子,dì zǐ,disciple
9,宏伟,hóng wěi,grand


In [62]:
ch = list(df[0])
py = list(df[1])
en = list(df[2])

samples = '\n'.join(['-'.join(sample) for sample in list(zip(ch,py,en))])
samples

"伟大-wěi dà-great\n兄弟-xiōng dì-brotherly\n弟-dì-variant of 悌[tì]\n弟兄-dì xiōng-comrade\n弟弟-dì di-younger brother\n内弟-nèi dì-wife's younger brother\n子弟-zǐ dì-people\n伟-wěi-big\n弟子-dì zǐ-disciple\n宏伟-hóng wěi-grand\n小弟-xiǎo dì-junior\n雄伟-xióng wěi-grand\n徒弟-tú dì-apprentice"

In [64]:
print(samples)

伟大-wěi dà-great
兄弟-xiōng dì-brotherly
弟-dì-variant of 悌[tì]
弟兄-dì xiōng-comrade
弟弟-dì di-younger brother
内弟-nèi dì-wife's younger brother
子弟-zǐ dì-people
伟-wěi-big
弟子-dì zǐ-disciple
宏伟-hóng wěi-grand
小弟-xiǎo dì-junior
雄伟-xióng wěi-grand
徒弟-tú dì-apprentice
